The Iowa Democratic Party is uploading the results of Tuesday's caucus to https://results.thecaucuses.org.
I haven't found a source for the raw data, so we have to scrape it.

Unfortunatley, the HTML isn't the cleanest... (https://twitter.com/therriaultphd/status/1224833286599585793).

In [8]:
import io
import requests
import lxml.html
import pandas as pd

url = "https://results.thecaucuses.org"
r = requests.get(url)

root = lxml.html.parse(io.StringIO(r.text)).getroot()

In [9]:
# Bennet, Biden, etc.
head = root.find_class("thead")[0]
header = [x.text or '' for x in list(head.iterchildren())]

# First Expression, Final Expression, SDE, ...
subhead = root.find_class("sub-head")[0]
subheader = [x.text or '' for x in list(subhead.iterchildren())]
columns = [f'{head}_{sub}'.strip('_') for head, sub in zip(header, subheader)]

In [3]:
counties = root.find_class("precinct-county")
county_names = [x[0].text for x in counties]
counties_data = root.find_class("precinct-data")
county = counties_data[0]
rows = []

In [4]:
for name, county in zip(county_names, counties_data):
    if len(county) > 1:
        # satellites only have a total
        county = county[:-1]

    for precinct in county:
        # exclude total
        rows.append((name,) + tuple(x.text for x in precinct))

In [5]:
df = pd.DataFrame(rows, columns=columns).set_index(['County', 'Precinct']).apply(pd.to_numeric)
df

Bennet_First Expression  \
County                 Precinct                                  
Adair                  1NW ADAIR                             0   
                       5GF GREENFIELD                        0   
Adams                  Adams 1                               0   
                       Adams 4                               0   
                       Adams 5                               0   
...                                                        ...   
Out of State Satellite Total                                 0   
CD1 Satellite          Total                                 0   
CD2 Satellite          Total                                 0   
CD3 Satellite          Total                                 0   
CD4 Satellite          Total                                 0   

                                       Final Expression  SDE  \
County                 Precinct                                
Adair                  1NW ADAIR                      0    0   
                       5GF GREENFIELD                 0    0   
Adams                  Adams 1                        0    0   
                       Adams 4                        0    0   
                       Adams 5                        0    0   
...                                                 ...  ...   
Out of State Satellite Total                          0    0   
CD1 Satellite          Total                          0    0   
CD2 Satellite          Total                          0    0   
CD3 Satellite          Total                          0    0   
CD4 Satellite          Total                          0    0   

                                       Biden_First Expression  \
County                 Precinct                                 
Adair                  1NW ADAIR                            6   
                       5GF GREENFIELD                       8   
Adams                  Adams 1                              5   
                       Adams 4                              6   
                       Adams 5                              4   
...                                                       ...   
Out of State Satellite Total                                0   
CD1 Satellite          Total                                0   
CD2 Satellite          Total                                0   
CD3 Satellite          Total                                0   
CD4 Satellite          Total                                0   

                                       Final Expression     SDE  \
County                 Precinct                                   
Adair                  1NW ADAIR                      6  0.0784   
                       5GF GREENFIELD                 0  0.0000   
Adams                  Adams 1                        5  0.0857   
                       Adams 4                        6  0.1714   
                       Adams 5                        4  0.0857   
...                                                 ...     ...   
Out of State Satellite Total                          0  0.0000   
CD1 Satellite          Total                          0  0.0000   
CD2 Satellite          Total                          0  0.0000   
CD3 Satellite          Total                          0  0.0000   
CD4 Satellite          Total                          0  0.0000   

                                       Bloomberg_First Expression  \
County                 Precinct                                     
Adair                  1NW ADAIR                                0   
                       5GF GREENFIELD                           0   
Adams                  Adams 1                                  0   
                       Adams 4                                  0   
                       Adams 5                                  0   
...                                                           ...   
Out of State Satellite Total                                    0   
CD1 Sate

In [11]:
df.columns

Index(['Bennet_First Expression', 'Final Expression', 'SDE',
       'Biden_First Expression', 'Final Expression', 'SDE',
       'Bloomberg_First Expression', 'Final Expression', 'SDE',
       'Buttigieg_First Expression', 'Final Expression', 'SDE',
       'Delaney_First Expression', 'Final Expression', 'SDE',
       'Gabbard_First Expression', 'Final Expression', 'SDE',
       'Klobuchar_First Expression', 'Final Expression', 'SDE',
       'Patrick_First Expression', 'Final Expression', 'SDE',
       'Sanders_First Expression', 'Final Expression', 'SDE',
       'Steyer_First Expression', 'Final Expression', 'SDE',
       'Warren_First Expression', 'Final Expression', 'SDE',
       'Yang_First Expression', 'Final Expression', 'SDE',
       'Other_First Expression', 'Final Expression', 'SDE',
       'Uncommitted_First Expression', 'Final Expression', 'SDE'],
      dtype='object')